In [1]:
import numpy as np
import pandas as pd
from fastapi import FastAPI, HTTPException
from keras.layers import LSTM, Dense, Dropout
from keras.models import Sequential
from pydantic import BaseModel
from sklearn.preprocessing import MinMaxScaler
import yfinance as yf

In [2]:
# df = pd.read_csv(r"C:\Users\VikashKumarChaudhary\Documents\LSTM\TCS1.csv")
# df1 = pd.read_csv(r"C:\Users\VikashKumarChaudhary\Documents\LSTM\RELIANCE1.csv")
# df2 = pd.read_csv(r"C:\Users\VikashKumarChaudhary\Documents\LSTM\ZOMATO1.csv")
# df3 = pd.read_csv(r"C:\Users\VikashKumarChaudhary\Documents\LSTM\TCS1.csv")
# df4 = pd.read_csv(r"C:\Users\VikashKumarChaudhary\Documents\LSTM\HDFC1.csv")

In [3]:
df1 = yf.download(tickers='RELIANCE.NS', period='475d', interval='1d')

df2 = yf.download(tickers='ZOMATO.NS', period='475d', interval='1d')

df3 = yf.download(tickers='TCS.NS', period='475d', interval='1d')

df4 = yf.download(tickers='HDFCBANK.NS', period='475d', interval='1d')

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [4]:
df1.to_csv(r"C:\Users\VikashKumarChaudhary\Documents\LSTM\API_R.csv")

In [5]:
df3


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2021-07-23,3230.000000,3241.550049,3206.649902,3212.850098,3077.407959,1776193
2021-07-26,3200.000000,3224.250000,3190.850098,3197.550049,3062.752930,1259611
2021-07-27,3200.000000,3210.000000,3161.149902,3182.949951,3048.768311,1121367
2021-07-28,3188.000000,3204.449951,3132.399902,3197.800049,3062.992676,1660654
2021-07-29,3200.250000,3225.449951,3190.300049,3195.800049,3061.076904,1731997
...,...,...,...,...,...,...
2023-06-15,3245.300049,3247.449951,3210.000000,3216.300049,3216.300049,2273810
2023-06-16,3197.000000,3199.399902,3156.000000,3174.899902,3174.899902,4156189
2023-06-19,3190.000000,3214.899902,3175.000000,3210.399902,3210.399902,1468186


In [6]:
df4.shape

(475, 6)

In [7]:
df = pd.concat([df1,df2,df3,df4],ignore_index = True)

In [8]:
df

,Open,High,Low,Close,Adj Close,Volume
0,2134.449951,2134.449951,2102.050049,2105.699951,2099.379395,4585938
1,2108.000000,2121.000000,2071.100098,2076.850098,2070.615967,5717830
2,2076.800049,2083.699951,2042.000000,2055.000000,2048.831543,4995516
3,2058.000000,2058.000000,2016.250000,2037.349976,2031.234497,5952355
4,2054.000000,2074.399902,2043.400024,2053.250000,2047.086792,6352886
...,...,...,...,...,...,...
1895,1604.000000,1605.949951,1579.000000,1582.050049,1582.050049,17688738
1896,1587.150024,1609.199951,1584.000000,1602.750000,1602.750000,7889636
1897,1611.000000,1614.250000,1598.349976,1604.150024,1604.150024,12414573
1898,1604.150024,1611.000000,1590.650024,1607.500000,1607.500000,10002813


In [9]:
# df['Date'] = pd.to_datetime(df['Date'])
# df = df.set_index('Date')

data = df.filter(["Open", "High", "Low", "Close"])
dataset = data.values

In [10]:
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(dataset)

In [11]:
seq_len = 60
train_size = int(len(scaled_data) * 0.8)
train_data = scaled_data[:train_size]
test_data = scaled_data[train_size - seq_len:]

In [12]:
x_train, y_train = [], []
for i in range(seq_len, len(train_data)):
    x_train.append(train_data[i - seq_len: i])
    y_train.append(train_data[i])

x_train, y_train = np.array(x_train), np.array(y_train)

In [13]:
model = Sequential()
model.add(LSTM(256, return_sequences=True, input_shape=(x_train.shape[1], x_train.shape[2])))
model.add(Dropout(0.4))
model.add(LSTM(256, return_sequences = False))
model.add(Dropout(0.2))
# model.add(LSTM(128))
# model.add(Dropout(0.2))
model.add(Dense(16, activation = 'linear'))
model.add(Dense(4))

In [14]:
model.compile(optimizer='adam', loss='mean_squared_error')
model.fit(x_train, y_train, batch_size=128, epochs=40,verbose=2)

Epoch 1/40
12/12 - 7s - loss: 0.0566 - 7s/epoch - 579ms/step
Epoch 2/40
12/12 - 4s - loss: 0.0090 - 4s/epoch - 354ms/step
Epoch 3/40
12/12 - 4s - loss: 0.0069 - 4s/epoch - 369ms/step
Epoch 4/40
12/12 - 4s - loss: 0.0057 - 4s/epoch - 365ms/step
Epoch 5/40
12/12 - 4s - loss: 0.0052 - 4s/epoch - 359ms/step
Epoch 6/40
12/12 - 4s - loss: 0.0054 - 4s/epoch - 374ms/step
Epoch 7/40
12/12 - 5s - loss: 0.0048 - 5s/epoch - 449ms/step
Epoch 8/40
12/12 - 5s - loss: 0.0049 - 5s/epoch - 445ms/step
Epoch 9/40
12/12 - 5s - loss: 0.0047 - 5s/epoch - 415ms/step
Epoch 10/40
12/12 - 5s - loss: 0.0044 - 5s/epoch - 426ms/step
Epoch 11/40
12/12 - 5s - loss: 0.0044 - 5s/epoch - 400ms/step
Epoch 12/40
12/12 - 5s - loss: 0.0043 - 5s/epoch - 408ms/step
Epoch 13/40
12/12 - 5s - loss: 0.0041 - 5s/epoch - 453ms/step
Epoch 14/40
12/12 - 5s - loss: 0.0039 - 5s/epoch - 420ms/step
Epoch 15/40
12/12 - 5s - loss: 0.0042 - 5s/epoch - 405ms/step
Epoch 16/40
12/12 - 5s - loss: 0.0039 - 5s/epoch - 410ms/step
Epoch 17/40
12/12

In [15]:
model.save(r"C:\Users\VikashKumarChaudhary\Documents\LSTM/Model_V3.2.h5")

In [16]:
# Load the test data
from keras.models import load_model
from sklearn.metrics import mean_squared_error

test_data = pd.read_csv(r"C:\Users\VikashKumarChaudhary\Documents\LSTM\zomato1.csv")
test_data['Date'] = pd.to_datetime(test_data['Date'])
test_data = test_data.set_index('Date')
test_dataset = test_data.filter(["Open", "High", "Low", "Close"]).values

scaler = MinMaxScaler(feature_range=(0, 1))
scaled_test_data = scaler.fit_transform(test_dataset)

seq_len = 80
x_test, y_test = [], []
for i in range(seq_len, len(scaled_test_data)):
    x_test.append(scaled_test_data[i - seq_len: i])
    y_test.append(scaled_test_data[i])

x_test, y_test = np.array(x_test), np.array(y_test)

model = load_model(r"C:\Users\VikashKumarChaudhary\Documents\LSTM/Model_V3.2.h5")

predictions = model.predict(x_test)

predictions = scaler.inverse_transform(predictions)
y_test = scaler.inverse_transform(y_test)

mse = mean_squared_error(y_test, predictions)

accuracy = np.sqrt(mse)

print("RMSE:", accuracy)


ValueError: in user code:

    File "C:\Users\VikashKumarChaudhary\AppData\Roaming\Python\Python310\site-packages\keras\engine\training.py", line 2169, in predict_function  *
        return step_function(self, iterator)
    File "C:\Users\VikashKumarChaudhary\AppData\Roaming\Python\Python310\site-packages\keras\engine\training.py", line 2155, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\Users\VikashKumarChaudhary\AppData\Roaming\Python\Python310\site-packages\keras\engine\training.py", line 2143, in run_step  **
        outputs = model.predict_step(data)
    File "C:\Users\VikashKumarChaudhary\AppData\Roaming\Python\Python310\site-packages\keras\engine\training.py", line 2111, in predict_step
        return self(x, training=False)
    File "C:\Users\VikashKumarChaudhary\AppData\Roaming\Python\Python310\site-packages\keras\utils\traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from Non